In [ ]:
!pip install opencv-python

In [ ]:
!pip install torchvision

In [ ]:
!pip install tensorflow keras opencv-python numpy

In [2]:
import pandas as pd
import numpy as np
import random

In [2]:
train_labels = pd.read_csv('Labels/TrainLabels.csv')

In [17]:
test_labels = pd.read_csv('Labels/TestLabels.csv')

In [18]:
test_labels['Engagement'].value_counts()

2    882
3    814
1     84
0      4
Name: Engagement, dtype: int64

In [30]:
random.sample(list(test_labels[test_labels['Engagement'] == 0]['ClipID']), 4)

['8264120240.avi', '5100351022.avi', '9877360133.avi', '5100342022.avi']

In [3]:
train_labels

,ClipID,Boredom,Engagement,Confusion,Frustration
0,1100011002.avi,0,2,0,0
1,1100011003.avi,0,2,0,0
2,1100011004.avi,0,3,0,0
3,1100011005.avi,0,3,0,0
4,1100011006.avi,0,3,0,0
...,...,...,...,...,...
5352,4599990246.avi,0,3,0,0
5353,4599990247.avi,0,3,0,0
5354,4599990248.avi,1,2,1,1
5355,4599990249.avi,0,3,0,0


In [20]:
random.sample(list(train_labels[train_labels['Engagement'] == 3]['ClipID']), 30)

['248510233.mp4',
 '1100142056.avi',
 '4140810154.avi',
 '2100572050.avi',
 '29042801630.mp4',
 '24851027.mp4',
 '303830184.mp4',
 '1813740127.avi',
 '3344630110.avi',
 '3100791061.avi',
 '4000181066.avi',
 '3100741063.avi',
 '1100011064.avi',
 '2408460151.avi',
 '3100791045.avi',
 '33446301107.avi',
 '1110032032.avi',
 '2408460276.avi',
 '3344630260.avi',
 '2100592044.avi',
 '248510273.mp4',
 '1100062009.avi',
 '3344630271.avi',
 '1813740238.avi',
 '303830236.mp4',
 '3100742011.avi',
 '3100752052.avi',
 '1100072012.avi',
 '3100721016.avi',
 '4599990137.avi']

In [5]:
random.sample(list(train_labels[train_labels['Engagement'] == 2]['ClipID']), 34)

['1100121049.avi',
 '3100771075.avi',
 '3422270142.avi',
 '2100552003.avi',
 '2000541035.avi',
 '3100631023.avi',
 '2100592070.avi',
 '2100602001.avi',
 '3503610133.avi',
 '1100121041.avi',
 '4110311068.avi',
 '1100042059.avi',
 '4110212024.avi',
 '3344630116.avi',
 '2100521031.avi',
 '3100812016.avi',
 '3100772025.avi',
 '2100602024.avi',
 '2100602004.avi',
 '3100771055.avi',
 '3100622026.avi',
 '1100071026.avi',
 '1100121033.avi',
 '2100572030.avi',
 '3100762016.avi',
 '2408460211.avi',
 '3344630171.avi',
 '4018350279.mp4',
 '3100751064.avi',
 '3422270251.avi',
 '1100052070.avi',
 '33702101600.mp4',
 '2100602003.avi',
 '1100161029.avi']

In [4]:
val_labels = pd.read_csv('Labels/ValidationLabels.csv')

In [5]:
val_labels['Engagement'].value_counts()

2    813
3    450
1    143
0     23
Name: Engagement, dtype: int64

In [16]:
# 10개씩 추출
random.sample(list(val_labels[val_labels['Engagement'] == 3]['ClipID']), 10)

['5000431050.avi',
 '5674960216.avi',
 '4100282024.avi',
 '4100321026.avi',
 '4000221017.avi',
 '4100201004.avi',
 '5674960276.avi',
 '567496019.avi',
 '4000221016.avi',
 '556463027.avi']

import cv2
cap = cv2.VideoCapture('DataSet/Train/110005/1100051002/1100051002.avi')
#num = 0
while (cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        cv2.imshow("frame", frame)
        #filepath = "snapshot/snapshot_" + str(num) + ".jpg"
        #cv2.imwrite(filepath, frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    #num += 1
cap.release()
cv2.destroyAllWindows()

import cv2
import sys

set_dir_images = []
humans = os.listdir('DataSet/Train/')
for human in humans:
    human_dir = "DataSet/Train/" + human + "/"
    videos = os.listdir(human_dir)
    for video in videos:
        video_dir = human_dir + video + "/"
        con = os.listdir(video_dir)[0]
        print(con)
        cap = cv2.VideoCapture(con)
        num = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if ret:
                cv2.imshow("frame", frame)
                filepath = "snapshot/snapshot_" + str(num) + ".jpg"
                cv2.imwrite(filepath, frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            num += 1

        cap.release()
        cv2.destroyAllWindows()

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import cv2
import os
from tqdm import tqdm
import tensorflow_datasets as tfds
import pandas as pd
import random


AUTOTUNE = tf.data.experimental.AUTOTUNE
batch_size = 32
np.random.seed(0)


class DataPreprocessing:
    def __init__(self,
                 IMG_HEIGHT=224,
                 IMG_WIDTH=224,
                 dataset_dir='DataSet/',
                 test_dir='Test/',
                 train_dir='Train/',
                 val_dir='Validation/',
                 labels_dir='Labels/',
                 test_label='TestLabels.csv',
                 train_label='TrainLabels.csv',
                 val_label='ValidationLabels.csv',
                 data_augmentation_flag=False,
                 max_frames=1
                 ):
        self.IMG_HEIGHT = IMG_HEIGHT
        self.IMG_WIDTH = IMG_WIDTH
        self.dataset_dir = dataset_dir
        self.train_dir = self.dataset_dir+train_dir
        self.test_dir = self.dataset_dir+test_dir
        self.val_dir = self.dataset_dir+val_dir
        self.labels_dir = labels_dir
        self.train_label_dir = self.labels_dir + train_label
        self.test_label_dir = self.labels_dir + test_label
        self.val_label_dir = self.labels_dir + val_label
        self.data_augmentation_flag = data_augmentation_flag
        self.max_frames = max_frames
        self.face_cascade = cv2.CascadeClassifier('dataset/haarcascade_frontalface_default.xml')

    def get_images_from_set_dir(self, setdir):
        '''
        Method to find all images in the tree folder
        '''
        set_dir_images = []
        humans = os.listdir(setdir)
        for human in humans:
            human_dir = setdir + human + "/"
            videos = os.listdir(human_dir)
            for video in videos:
                video_dir = human_dir + video + "/"    
                pictures = os.listdir(video_dir)
                pictures = random.sample(pictures, self.max_frames)
                for picture in pictures:
                    picture_dir = video_dir + picture
                    if picture.endswith(".jpg"):
                        set_dir_images.append(picture_dir)
                        
        return set_dir_images

    def get_labels_dataframe(self):
        '''
        Method to read pandas dataframe
        '''
        train_df = pd.read_csv(self.train_label_dir, sep=",")
        test_df = pd.read_csv(self.test_label_dir, sep=",")
        val_df = pd.read_csv(self.val_label_dir, sep=",")
        return train_df, test_df, val_df

    def resize(self, image):
        return cv2.resize(image, (self.IMG_HEIGHT, self.IMG_WIDTH), interpolation=cv2.INTER_AREA)

    def face_cropping(self, image):
        # Crop and resize
        faces = self.face_cascade.detectMultiScale(image, 1.3, 5)
        try:
            if faces != 0:
                x, y, w, h = faces[0]
                image = image[y:y+h, x:x+w]
        except:
            pass
        return self.resize(image)

    def random_crop(self, image, crop_height, crop_width):
        max_x = image.shape[1] - crop_width
        max_y = image.shape[0] - crop_height

        x = np.random.randint(0, max_x)
        y = np.random.randint(0, max_y)

        crop = image[y: y + crop_height, x: x + crop_width]

        return self.face_cropping(crop)


    def augment_image(self, image):
        '''
        Applies some augmentation techniques
        '''
        # Mirror flip
        flipped = tf.image.flip_left_right(image).numpy()
        # Transpose flip
        transposed = tf.image.transpose(image).numpy()
        # Saturation
        satured = tf.image.adjust_saturation(image, 3).numpy()
        # Brightness
        brightness = tf.image.adjust_brightness(image, 0.4).numpy()
        # Contrast
        contrast = tf.image.random_contrast(image, lower=0.0, upper=1.0).numpy()
        # Resize at the end
        images = [self.resize(image) for image in [flipped, transposed, satured, brightness, contrast]]
        return images


    def get_label_picture(self, image_path, label_df):
        error_ = False
        video = image_path.split("/")[-2]
        label_series = label_df.loc[((label_df['ClipID'] == video+'.avi') | (label_df['ClipID'] == video+'.mp4'))]
        try:
            index = label_series.index.values[0]
            label = np.array([label_series['Boredom'].get(index),
                              label_series['Engagement'].get(index),
                              label_series['Confusion'].get(index),
                              label_series['Frustration '].get(index)])
            label_one_hot = (label >= 1).astype(np.uint8)
        except:
            print('Error in label picture')
            print(image_path)
            label_one_hot = ''
            error_ = True
        return label_one_hot, error_

    def _int64_feature(self, value):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

    def _bytes_feature(self, value):
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

    def writeTfRecord(self, output_dir, data_augmentation=False):
        '''
        Method to write tfrecord
        '''
        # open the TFRecords file
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Read dataframes
        train_df, test_df, val_df = self.get_labels_dataframe()

        # Objects to iterate
        objs = [('train', self.train_dir, train_df),
                ('test', self.test_dir, test_df),
                ('val', self.val_dir, val_df)]

        for name, dataset, label_df in tqdm(objs):
            # Open Writer
            writer = tf.io.TFRecordWriter(output_dir+name+'.tfrecords')
            # Get all the images of a set
            images_path = self.get_images_from_set_dir(dataset)
            for image_path in tqdm(images_path, total=len(images_path)):
                # Read the image from path
                img = cv2.imread(image_path)[..., ::-1]
                img = self.face_cropping(img)
                # Read the label
                label, error_ = self.get_label_picture(image_path, label_df)
                if error_:
                    continue
                # Create a feature
                if data_augmentation:
                    images = self.augment_image(img)
                else:
                    images = img
                for image in images:
                    feature = {'label': self._bytes_feature(tf.compat.as_bytes(label.tostring())),
                               'image': self._bytes_feature(tf.compat.as_bytes(image.tostring()))}
                    # Create an example protocol buffer
                    example = tf.train.Example(features=tf.train.Features(feature=feature))

                    # Serialize to string and write on the file
                    writer.write(example.SerializeToString())
            writer.close()

    def decode(self, serialized_example):
        """
        Parses an image and label from the given `serialized_example`.
        It is used as a map function for `dataset.map`
        """
        IMAGE_SHAPE = (self.IMG_HEIGHT, self.IMG_WIDTH, 3)

        # 1. define a parser
        features = tf.io.parse_single_example(
            serialized_example,
            # Defaults are not specified since both keys are required.
            features={
                'image': tf.io.FixedLenFeature([], tf.string),
                'label': tf.io.FixedLenFeature([], tf.string),
            })

        # 2. Convert the data
        image = tf.io.decode_raw(features['image'], tf.uint8)
        label = tf.io.decode_raw(features['label'], tf.uint8)

        # Cast
        label = tf.cast(label, tf.float32)

        # 3. reshape
        image = tf.convert_to_tensor(tf.reshape(image, IMAGE_SHAPE))

        return image, label


if __name__ == '__main__':
    preprocessing_class = DataPreprocessing()
    # Write tf recordfloat32
    preprocessing_class.writeTfRecord('tfrecords/', data_augmentation=True)

    # Read TfRecord
    tfrecord_path = 'tfrecords/train.tfrecords'
    dataset = tf.data.TFRecordDataset(tfrecord_path)

    # Parse the record into tensors with map.
    # map takes a Python function and applies it to every sample.
    dataset = dataset.map(preprocessing_class.decode)

    # Divide in batch
    dataset = dataset.batch(batch_size)

    # Create an iterator
    iterator = iter(dataset)

    # Element of iterator
    #a = iterator.get_next()

In [ ]:
def network():
    model = tf.keras.Sequential()
    model.add(kl.InputLayer(input_shape=(224, 224, 3)))
    # First conv block
    model.add(kl.Conv2D(filters=96, kernel_size=7, padding='same', strides=2))
    model.add(tf.keras.layers.ReLU())
    model.add(kl.MaxPooling2D(pool_size=(3, 3)))
    # Second conv block
    model.add(kl.Conv2D(filters=256, kernel_size=5, padding='same', strides=1))
    model.add(tf.keras.layers.ReLU())
    model.add(kl.MaxPooling2D(pool_size=(2, 2)))
    # Third-Fourth-Fifth conv block
    for i in range(3):
        model.add(kl.Conv2D(filters=512, kernel_size=3, padding='same', strides=1))
        model.add(tf.keras.layers.ReLU())
    model.add(kl.MaxPooling2D(pool_size=(3, 3)))
    # Flatten
    model.add(kl.Flatten())
    # First FC
    model.add(kl.Dense(4048))
    # Second Fc
    model.add(kl.Dense(4048))
    # Third FC
    model.add(kl.Dense(4))
    # Softmax at the end
    model.add(kl.Softmax())

    return model

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as kl
import tensorflow.keras.losses
from tensorflow.keras.applications.resnet50 import ResNet50


# In[2]:


#from daisee_data_preprocessing import DataPreprocessing
import datetime
import os
from tqdm import tqdm


# In[3]:


BATCH_SIZE = 32
LR = 0.005
EPOCHS = 100


# In[4]:


current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
checkpoint_dir = 'checkpoints/'
log_dir = 'logs/'


# In[5]:


### This part works for setting up space in gpu
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 1 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 2)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)


# In[6]:


preprocessing_class = DataPreprocessing()


# In[7]:


# Open train set
tfrecord_path = 'tfrecords/train.tfrecords'
train_set = tf.data.TFRecordDataset(tfrecord_path)
# Parse the record into tensors with map.
train_set = train_set.map(preprocessing_class.decode)
train_set = train_set.shuffle(1)
train_set = train_set.batch(BATCH_SIZE)


# In[8]:


# Open test set
tfrecord_path = 'tfrecords/test.tfrecords'
test_set = tf.data.TFRecordDataset(tfrecord_path)
# Parse the record into tensors with map.
test_set = test_set.map(preprocessing_class.decode)
test_set = test_set.shuffle(1)
test_set = test_set.batch(BATCH_SIZE)


# In[9]:


# Open val set
tfrecord_path = 'tfrecords/val.tfrecords'
val_set = tf.data.TFRecordDataset(tfrecord_path)
# Parse the record into tensors with map.
val_set = val_set.map(preprocessing_class.decode)
val_set = val_set.shuffle(1)
val_set = val_set.batch(BATCH_SIZE)


# In[10]:


def create_log_dir(log_dir, checkpoint_dir):
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)
    if not os.path.exists(checkpoint_dir):
        os.mkdir(checkpoint_dir)


# In[11]:


def network():
    model = tf.keras.Sequential()
    model.add(kl.InputLayer(input_shape=(224, 224, 3)))
    # First conv block
    model.add(kl.Conv2D(filters = 96, kernel_size=7, padding='same', strides=2))
    model.add(tf.keras.layers.ReLU())
    model.add(kl.MaxPooling2D(pool_size=(3, 3)))
    # Second conv block
    model.add(kl.Conv2D(filters = 256, kernel_size=5, padding='same', strides=1))
    model.add(tf.keras.layers.ReLU())
    model.add(kl.MaxPooling2D(pool_size=(2, 2)))
    # Third-Fourth-Fifth conv block
    for i in range(3):
        model.add(kl.Conv2D(filters = 512, kernel_size=3, padding='same', strides=1))
        model.add(tf.keras.layers.ReLU())
    model.add(kl.MaxPooling2D(pool_size=(3, 3)))
    # Flatten
    model.add(kl.Flatten())
    # First FC 
    model.add(kl.Dense(4048))
    # Second Fc
    model.add(kl.Dense(4048))
    # Third FC
    model.add(kl.Dense(4))
    # Softmax at the end
    model.add(kl.Softmax())
    
    return model


# In[12]:


model = network()


# In[13]:


'''
https://keras.io/guides/writing_a_training_loop_from_scratch/
Compile into a static graph any function that take tensors as input to apply global performance optimizations.
'''


# In[14]:


@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss_value = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    # Track progress
    train_loss_avg.update_state(loss_value)
    train_accuracy.update_state(y, logits)
    return loss_value


# In[15]:


@tf.function
def test_step(x, y, set_name):
    logits = model(x)
    if set_name == 'val':
        val_accuracy.update_state(y, logits)
    else:
        test_accuracy.update_state(y, logits)


# In[16]:


optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
create_log_dir(log_dir, checkpoint_dir)
train_summary_writer = tf.summary.create_file_writer(log_dir)


# In[17]:


train_loss_avg = tf.keras.metrics.Mean()
train_accuracy = tf.keras.metrics.MeanAbsoluteError()
val_accuracy = tf.keras.metrics.MeanAbsoluteError()
test_accuracy = tf.keras.metrics.MeanAbsoluteError()


# In[ ]:


for epoch in range(EPOCHS):
    # Training loop
    for x_batch_train, y_batch_train in tqdm(train_set, total=1517):
        # Do step
        loss_value = train_step(x_batch_train, y_batch_train)
        
    # Test on validation set
    for x_batch_val, y_batch_val in val_set:
        test_step(x_batch_val, y_batch_val, 'val')
    
    # Reset training metrics at the end of each epoch
    train_acc = train_accuracy.result()
    train_accuracy.reset_states()
    val_acc = val_accuracy.result()
    val_accuracy.reset_states()
    
    with train_summary_writer.as_default():
        tf.summary.scalar('Train loss', train_loss_avg.result(), step=epoch)
        tf.summary.scalar('Train MAE', train_acc, step=epoch)
        tf.summary.scalar('Val MAE', val_acc, step=epoch)
        
    if epoch % 10 == 0:
        tf.keras.models.save_model(model, '{}/Epoch_{}_model.hp5'.format(checkpoint_dir, str(epoch)), save_format="h5")


# In[ ]:


# Test on validation set
for x_batch_test, y_batch_test in test_set:
    test_step(x_batch_test, y_batch_test, 'test')
test_set_acc = test_accuracy.result()